In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

In [7]:
df_jan_path = "../data/raw/yellow_tripdata_2016-01.csv"
df_feb_path = "../data/raw/yellow_tripdata_2016-02.csv"
df_mar_path = "../data/raw/yellow_tripdata_2016-03.csv"

# load the dataframes

df_jan = dd.read_csv(df_jan_path, assume_missing=True,usecols=['trip_distance', 'tpep_pickup_datetime', 'pickup_longitude',
       'pickup_latitude','dropoff_longitude', 'dropoff_latitude', 'fare_amount'],parse_dates=["tpep_pickup_datetime"])
df_feb = dd.read_csv(df_feb_path, assume_missing=True,usecols=['trip_distance', 'tpep_pickup_datetime', 'pickup_longitude',
       'pickup_latitude','dropoff_longitude', 'dropoff_latitude', 'fare_amount'],parse_dates=["tpep_pickup_datetime"])
df_mar = dd.read_csv(df_mar_path, assume_missing=True,usecols=['trip_distance', 'tpep_pickup_datetime', 'pickup_longitude',
       'pickup_latitude','dropoff_longitude', 'dropoff_latitude', 'fare_amount'],parse_dates=["tpep_pickup_datetime"])

In [8]:
df=dd.concat([df_jan,df_feb,df_mar],axis=0)

In [9]:
df

,tpep_pickup_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount
npartitions=82,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [27]:
min_latitude = 40.60
max_latitude = 40.85

min_longitude = -74.05
max_longitude = -73.70

min_fare_amount_val = 0.50
max_fare_amount_val = 81.0

min_trip_distance_val = 0.25
max_trip_distance_val = 24.43

In [28]:
df = df.loc[(df["pickup_latitude"].between(min_latitude, max_latitude, inclusive="both")) & 
(df["pickup_longitude"].between(min_longitude, max_longitude, inclusive="both")) & 
(df["dropoff_latitude"].between(min_latitude, max_latitude, inclusive="both")) & 
(df["dropoff_longitude"].between(min_longitude, max_longitude, inclusive="both")), :]

In [11]:
data_path = "../data/interim/location_data.csv"

In [13]:
df_reader = pd.read_csv(data_path, chunksize=100000, usecols=["pickup_latitude","pickup_longitude"])

In [29]:
df = df.loc[(df["fare_amount"].between(min_fare_amount_val,max_fare_amount_val,inclusive="both")) & 
(df["trip_distance"].between(min_trip_distance_val,max_trip_distance_val,inclusive="both"))]

In [30]:
df.head()

,tpep_pickup_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount
0,2016-01-01,1.10,-73.990372,40.734695,-73.981842,40.732407,7.5
1,2016-01-01,4.90,-73.980782,40.729912,-73.944473,40.716679,18.0
2,2016-01-01,10.54,-73.984550,40.679565,-73.950272,40.788925,33.0
3,2016-01-01,4.75,-73.993469,40.718990,-73.962242,40.657333,16.5
4,2016-01-01,1.76,-73.960625,40.781330,-73.977264,40.758514,8.0


In [32]:
df.head()

,tpep_pickup_datetime,pickup_longitude,pickup_latitude
0,2016-01-01,-73.990372,40.734695
1,2016-01-01,-73.980782,40.729912
2,2016-01-01,-73.984550,40.679565
3,2016-01-01,-73.993469,40.718990
4,2016-01-01,-73.960625,40.781330


In [31]:
df=df.drop(columns=["trip_distance","fare_amount","dropoff_longitude","dropoff_latitude"])

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans

In [17]:
scaler=StandardScaler()
mini_batch=MiniBatchKMeans(n_clusters=30,n_init=10,random_state=42)

In [18]:
for chunk in df_reader:
    scaler.partial_fit(chunk)
    
   
    

In [19]:
scaler.mean_

array([ 40.75107121, -73.9740618 ])

In [20]:
df_reader = pd.read_csv(data_path, chunksize=100000, usecols=["pickup_latitude","pickup_longitude"])

In [21]:

for chunk in df_reader:
    scaled_chunk=scaler.transform(chunk)
    mini_batch.partial_fit(scaled_chunk)
    
   

In [23]:
dfs=dd.read_csv(data_path)

In [35]:
dfs.head(10)

,pickup_latitude,pickup_longitude
0,40.734695,-73.990372
1,40.729912,-73.980782
2,40.679565,-73.984550
3,40.718990,-73.993469
4,40.781330,-73.960625
5,40.743050,-73.980118
6,40.719990,-73.994057
7,40.744614,-73.979424
8,40.723896,-73.998344
9,40.744919,-74.006149


In [38]:
dfs.tail(10)

,pickup_latitude,pickup_longitude
1749143,40.641743,-73.789223
1749144,40.760170,-73.967552
1749145,40.748898,-73.937790
1749146,40.646561,-73.790436
1749147,40.761150,-73.986847
1749148,40.644451,-73.790565
1749149,40.641483,-73.788055
1749150,40.646736,-73.789154
1749151,40.774471,-73.977356
1749152,40.645096,-73.798485


In [43]:
df.head(10)

,tpep_pickup_datetime,pickup_longitude,pickup_latitude
0,2016-01-01 00:00:00,-73.990372,40.734695
1,2016-01-01 00:00:00,-73.980782,40.729912
2,2016-01-01 00:00:00,-73.984550,40.679565
3,2016-01-01 00:00:00,-73.993469,40.718990
4,2016-01-01 00:00:00,-73.960625,40.781330
5,2016-01-01 00:00:00,-73.980118,40.743050
6,2016-01-01 00:00:00,-73.994057,40.719990
7,2016-01-01 00:00:01,-73.979424,40.744614
9,2016-01-01 00:00:02,-73.998344,40.723896
10,2016-01-01 00:00:03,-74.006149,40.744919


In [45]:
location_subset=df[["pickup_latitude","pickup_longitude"]]

In [ ]:
location_subset.head(10) 

,pickup_latitude,pickup_longitude
0,40.734695,-73.990372
1,40.729912,-73.980782
2,40.679565,-73.984550
3,40.718990,-73.993469
4,40.781330,-73.960625
5,40.743050,-73.980118
6,40.719990,-73.994057
7,40.744614,-73.979424
9,40.723896,-73.998344
10,40.744919,-74.006149


In [47]:
scaled_location_subset=scaler.transform(location_subset)

In [54]:
reg=mini_batch.predict(scaled_location_subset)

In [56]:
type(reg)

numpy.ndarray

In [55]:
reg.shape

(33234199,)

In [51]:
from sklearn import set_config

In [53]:
set_config(transform_output="pandas")

In [59]:
reg=pd.DataFrame(reg)

In [60]:
reg

,0
0,7
1,26
2,9
3,10
4,8
...,...
33234194,3
33234195,3
33234196,3
33234197,19


In [62]:
df

,tpep_pickup_datetime,pickup_longitude,pickup_latitude
npartitions=82,,,
,datetime64[ns],float64,float64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [69]:
type(reg)

pandas.core.frame.DataFrame

In [72]:
df=df.assign(Region=reg[0])

In [73]:
df.head(10)

,tpep_pickup_datetime,pickup_longitude,pickup_latitude,Region
13,2016-02-08 15:53:52,-73.976135,40.776302,19
43,2016-02-08 15:53:58,-73.968178,40.787212,11
53,2016-02-08 15:53:59,-73.870712,40.773788,1
462,2016-02-08 15:55:30,-73.983017,40.756241,11
541,2016-02-08 15:55:50,-73.960060,40.773586,21
562,2016-02-08 15:55:55,-73.863930,40.769978,27
808,2016-02-08 15:56:49,-73.964279,40.767914,8
950,2016-02-08 15:57:20,-73.990555,40.756882,15
1046,2016-02-08 15:57:39,-73.990440,40.765732,1
1182,2016-02-08 15:58:08,-73.956100,40.781288,22


## Analysing the TimeSeries data

In [74]:
time_series_data=df.drop(columns=["pickup_longitude","pickup_latitude"])

In [79]:
time_series_data=time_series_data.compute()

In [81]:
save_path = "../data/interim/time_series.csv"

time_series_data.to_csv(save_path, index=False)

In [2]:
time_series_path = "../data/interim/time_series.csv"

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

In [18]:
time_series_data=pd.read_csv(time_series_path,parse_dates=["tpep_pickup_datetime"])

In [19]:
time_series_data.set_index("tpep_pickup_datetime",inplace=True)

In [15]:
time_series_data

,Region
tpep_pickup_datetime,
2016-01-10 19:00:07,19
2016-01-10 19:00:22,11
2016-01-10 19:00:24,1
2016-01-10 19:03:31,11
2016-01-10 19:04:10,21
...,...
2016-02-01 01:30:57,14
2016-02-01 01:32:21,22
2016-02-01 01:33:11,27


In [16]:
time_series_data.index

DatetimeIndex(['2016-01-10 19:00:07', '2016-01-10 19:00:22',
               '2016-01-10 19:00:24', '2016-01-10 19:03:31',
               '2016-01-10 19:04:10', '2016-01-10 19:04:22',
               '2016-01-10 19:06:11', '2016-01-10 19:07:36',
               '2016-01-10 19:08:18', '2016-01-10 19:09:23',
               ...
               '2016-02-01 01:29:14', '2016-02-01 01:30:07',
               '2016-02-01 01:30:38', '2016-02-01 01:30:47',
               '2016-02-01 01:30:52', '2016-02-01 01:30:57',
               '2016-02-01 01:32:21', '2016-02-01 01:33:11',
               '2016-02-01 01:36:43', '2016-02-01 01:37:02'],
              dtype='datetime64[ns]', name='tpep_pickup_datetime', length=33234199, freq=None)

In [17]:
time_series_data.resample("15min").count()

,Region
tpep_pickup_datetime,
2016-01-01 00:00:00,3927
2016-01-01 00:15:00,6460
2016-01-01 00:30:00,6755
2016-01-01 00:45:00,6906
2016-01-01 01:00:00,6963
...,...
2016-03-31 22:45:00,5731
2016-03-31 23:00:00,5664
2016-03-31 23:15:00,5395


In [20]:
reg_grp=time_series_data.groupby("Region")

In [21]:
resampl_data=reg_grp['Region'].resample("15min").count()

In [22]:
resampl_data.name="Pickups"

In [23]:
resample_data=resampl_data.reset_index(level=0)

In [24]:
resample_data

,Region,Pickups
tpep_pickup_datetime,,
2016-01-01 00:00:00,0,57
2016-01-01 00:15:00,0,113
2016-01-01 00:30:00,0,146
2016-01-01 00:45:00,0,162
2016-01-01 01:00:00,0,182
...,...,...
2016-03-31 22:45:00,29,13
2016-03-31 23:00:00,29,43
2016-03-31 23:15:00,29,24


In [98]:
resample_data[resample_data["Pickups"] == 0]

,Region,Pickups
tpep_pickup_datetime,,
2016-01-12 04:15:00,0,0
2016-01-23 15:15:00,0,0
2016-01-23 15:45:00,0,0
2016-01-23 16:45:00,0,0
2016-01-23 17:00:00,0,0
...,...,...
2016-03-15 04:00:00,29,0
2016-03-17 04:45:00,29,0
2016-03-27 05:15:00,29,0


In [25]:
resample_data["Pickups"].replace({0:10},inplace=True)

C:\Users\Satyajit Samal\AppData\Local\Temp\ipykernel_26008\4173565344.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  resample_data["Pickups"].replace({0:10},inplace=True)


## Smoothing Timeseries data

In [26]:
from sklearn.metrics import mean_absolute_percentage_error

In [101]:
window_values = list(range(3,11,1))
window_values

[3, 4, 5, 6, 7, 8, 9, 10]

In [104]:
def calculate_best_window_value(windows):
    for window in windows:
        ind = window - 1
        y_pred = resample_data['Pickups'].rolling(window=window).mean().values[ind:]
        y = resample_data['Pickups'].values[ind:]
        error = mean_absolute_percentage_error(y, y_pred)
        print(f"For window value {window}, the MAPE is {error:.2f}")

In [105]:
calculate_best_window_value(window_values)

For window value 3, the MAPE is 0.22
For window value 4, the MAPE is 0.28
For window value 5, the MAPE is 0.33
For window value 6, the MAPE is 0.37
For window value 7, the MAPE is 0.42
For window value 8, the MAPE is 0.46
For window value 9, the MAPE is 0.50
For window value 10, the MAPE is 0.54


## Applying EWMA

In [108]:
resample_data["Pickups"].ewm(0.9).mean()

tpep_pickup_datetime
2016-01-01 00:00:00     57.000000
2016-01-01 00:15:00     95.000000
2016-01-01 00:30:00    125.034258
2016-01-01 00:45:00    145.521332
2016-01-01 01:00:00    165.189673
                          ...    
2016-03-31 22:45:00     17.433698
2016-03-31 23:00:00     30.889646
2016-03-31 23:15:00     27.263517
2016-03-31 23:30:00     19.230087
2016-03-31 23:45:00     21.214252
Name: Pickups, Length: 262080, dtype: float64

In [27]:
alphas=np.arange(0.1,1.0,0.1)

In [28]:
alphas

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [29]:
def calculating_ewm(alphas):
    y=resample_data["Pickups"]
    for i in alphas:
        y_pred=resample_data["Pickups"].ewm(i).mean()
        
        mape=mean_absolute_percentage_error(y,y_pred)
        print(f"For alpha {i:.1f} MAPE :{mape:.2}")

In [30]:
calculating_ewm(alphas)

For alpha 0.1 MAPE :0.028
For alpha 0.2 MAPE :0.052
For alpha 0.3 MAPE :0.073
For alpha 0.4 MAPE :0.091
For alpha 0.5 MAPE :0.11
For alpha 0.6 MAPE :0.12
For alpha 0.7 MAPE :0.14
For alpha 0.8 MAPE :0.15
For alpha 0.9 MAPE :0.17


In [32]:
resample_data["Avg_pickups"]=resample_data["Pickups"].ewm(alpha=0.4).mean().round()

In [33]:
resample_data

,Region,Pickups,Avg_pickups
tpep_pickup_datetime,,,
2016-01-01 00:00:00,0,57,57.0
2016-01-01 00:15:00,0,113,92.0
2016-01-01 00:30:00,0,146,120.0
2016-01-01 00:45:00,0,162,139.0
2016-01-01 01:00:00,0,182,158.0
...,...,...,...
2016-03-31 22:45:00,29,13,19.0
2016-03-31 23:00:00,29,43,29.0
2016-03-31 23:15:00,29,24,27.0


In [34]:
resample_data_path='../data/interim/final_data.csv'
resample_data.to_csv(resample_data_path,index=True)